In [1]:
#pip install mediapipe --user

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

In [4]:
# train_gesture_model.py


# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Parameters
GESTURES = ['fist', 'palm', 'peace', 'thumbs_up']
NUM_SAMPLES = 100

data = []
labels = []

cap = cv2.VideoCapture(0)
gesture_index = 0
sample_count = 0
collecting = False

def extract_both_hand_landmarks(results):
    all_landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                all_landmarks.extend([lm.x, lm.y, lm.z])
    while len(all_landmarks) < 126:
        all_landmarks.extend([0.0] * 3)
    return all_landmarks

print("Press 'R' to start collecting each gesture.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    if collecting and sample_count < NUM_SAMPLES:
        landmarks = extract_both_hand_landmarks(results)
        if landmarks:
            data.append(landmarks)
            labels.append(GESTURES[gesture_index])
            sample_count += 1

    cv2.putText(frame, f'Gesture: {GESTURES[gesture_index]}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f'Samples: {sample_count}/{NUM_SAMPLES}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    if not collecting:
        cv2.putText(frame, 'Press R to start collecting', (10, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 200, 255), 2)

    cv2.imshow("Collecting Gesture Data", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('r'):
        if gesture_index < len(GESTURES):
            collecting = True
            print(f"Started collecting for: {GESTURES[gesture_index]}")

    if sample_count >= NUM_SAMPLES:
        print(f"Finished collecting: {GESTURES[gesture_index]}")
        collecting = False
        gesture_index += 1
        sample_count = 0
        if gesture_index >= len(GESTURES):
            print("✅ All gestures collected!")
            break

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Train model
print("Training model...")
X = np.array(data)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = RandomForestClassifier()
model.fit(X_train, y_train)

print("Model accuracy:", model.score(X_test, y_test))

# Save model
with open('hand_gesture_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✅ Model saved as 'hand_gesture_model.pkl'")

Press 'R' to start collecting each gesture.
Started collecting for: fist
Finished collecting: fist
Started collecting for: palm
Finished collecting: palm
Started collecting for: peace
Finished collecting: peace
Started collecting for: thumbs_up
Finished collecting: thumbs_up
✅ All gestures collected!
Training model...
Model accuracy: 1.0
✅ Model saved as 'hand_gesture_model.pkl'
